In [ ]:
!pip install tensorflow opencv-python

In [ ]:
!unzip -q /content/drive/MyDrive/reflection_removal_dataset/Dataset.zip -d /content/Dataset

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import cv2
import os

# Define UNet model
def unet_model(input_shape=(256, 256, 3)):
    inputs = layers.Input(input_shape)

    # Encoder
    c1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    c1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c1)
    p1 = layers.MaxPooling2D((2, 2))(c1)

    c2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(p1)
    c2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c2)
    p2 = layers.MaxPooling2D((2, 2))(c2)

    c3 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(p2)
    c3 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(c3)
    p3 = layers.MaxPooling2D((2, 2))(c3)

    # Bottleneck
    c4 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(p3)
    c4 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(c4)

    # Decoder
    u5 = layers.Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(c4)
    u5 = layers.concatenate([u5, c3])
    c5 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(u5)
    c5 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(c5)

    u6 = layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = layers.concatenate([u6, c2])
    c6 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(u6)
    c6 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c6)

    u7 = layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = layers.concatenate([u7, c1])
    c7 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(u7)
    c7 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c7)

    outputs = layers.Conv2D(3, (1, 1), activation='sigmoid')(c7)

    model = models.Model(inputs, outputs)
    return model

# Compile the model
model = unet_model()
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
model.summary()

# Data preparation
def load_images_from_folder(folder, target_size=(256, 256)):
    images = []
    for filename in sorted(os.listdir(folder)):
        img_path = os.path.join(folder, filename)
        img = cv2.imread(img_path)
        if img is not None:
            img = cv2.resize(img, target_size)
            img = img / 255.0  # Normalizing the image
            images.append(img)
    return np.array(images)

# Define paths
base_dir = 'Dataset/Dataset'
clear_dir = os.path.join(base_dir, 'clear')
reflection_dir = os.path.join(base_dir, 'img with reflection')

# Load datasets
X_train = load_images_from_folder(reflection_dir)
Y_train = load_images_from_folder(clear_dir)

# Check the shapes of loaded data
print(f"Loaded {len(X_train)} images with reflections.")
print(f"Loaded {len(Y_train)} clear images.")

# Ensure dataset size is consistent
assert len(X_train) == len(Y_train), "Mismatch between input and output datasets."

# Training parameters
batch_size = 16
num_epochs = 50
steps_per_epoch = len(X_train) // batch_size

# Custom data generator
def data_generator(X, Y, batch_size):
    while True:
        for i in range(0, len(X), batch_size):
            X_batch = X[i:i+batch_size]
            Y_batch = Y[i:i+batch_size]
            yield (np.array(X_batch), np.array(Y_batch))

# Create generators
train_gen = data_generator(X_train, Y_train, batch_size)

# Train the model
history = model.fit(train_gen, epochs=num_epochs, steps_per_epoch=steps_per_epoch)

# Save the model
model.save('reflection_removal_unet.h5')


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 256, 256, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d (Conv2D)           │ (None, 256, 256, 64)   │          1,792 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_1 (Conv2D)         │ (None, 256, 256, 64)   │         36,928 │ conv2d[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d             │ (None, 128, 128, 64)   │              0 │ conv2d_1[0][0]         │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_2 (Conv2D)         │ (None, 128, 128, 128)  │         73,856 │ max_pooling2d[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_3 (Conv2D)         │ (None, 128, 128, 128)  │        147,584 │ conv2d_2[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_1           │ (None, 64, 64, 128)    │              0 │ conv2d_3[0][0]         │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_4 (Conv2D)         │ (None, 64, 64, 256)    │        295,168 │ max_pooling2d_1[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_5 (Conv2D)         │ (None, 64, 64, 256)    │        590,080 │ conv2d_4[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_2           │ (None, 32, 32, 256)    │              0 │ conv2d_5[0][0]         │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_6 (Conv2D)         │ (None, 32, 32, 512)    │      1,180,160 │ max_pooling2d_2[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_7 (Conv2D)         │ (None, 32, 32, 512)    │      2,359,808 │ conv2d_6[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_transpose          │ (None, 64, 64, 256)    │        524,544 │ conv2d_7[0][0]         │
│ (Conv2DTranspose)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate (Concatenate) │ (None, 64, 64, 512)    │              0 │ conv2d_transpose[0][0… │
│                           │                        │                │ conv2d_5[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_8 (Conv2D)         │ (None, 64, 64, 256)    │      1,179,904 │ concatenate[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_9 (Conv2D)         │ (None, 64, 64, 256)    │        590,080 │ conv2d_8[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_transpose_1        │ (None, 128, 128, 128)  │        131,200 │ conv2d_9[0][0]         │
│ (Conv2DTranspose)    

 Total params: 7,697,475 (29.36 MB)

 Trainable params: 7,697,475 (29.36 MB)

 Non-trainable params: 0 (0.00 B)

Loaded 499 images with reflections.
Loaded 499 clear images.
Epoch 1/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 84s 614ms/step - accuracy: 0.5499 - loss: 0.0564
Epoch 2/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 41s 592ms/step - accuracy: 0.6475 - loss: 0.0164
Epoch 3/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 18s 578ms/step - accuracy: 0.6545 - loss: 0.0129
Epoch 4/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 18s 580ms/step - accuracy: 0.7199 - loss: 0.0102
Epoch 5/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 18s 578ms/step - accuracy: 0.7888 - loss: 0.0093
Epoch 6/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 18s 576ms/step - accuracy: 0.7799 - loss: 0.0088
Epoch 7/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 18s 576ms/step - accuracy: 0.7615 - loss: 0.0088
Epoch 8/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 18s 576ms/step - accuracy: 0.7732 - loss: 0.0090
Epoch 9/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 18s 579ms/step - accuracy: 0.7661 - loss: 0.0075
Epoch 10/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 18s 578ms/step - accuracy: 0.7871 - loss: 0.0071
Epoch 11/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 18s 578ms/step - accuracy: 0.

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import cv2
import os

# Define UNet model
def unet_model(input_shape=(256, 256, 3)):
    inputs = layers.Input(input_shape)

    # Encoder
    c1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    c1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c1)
    p1 = layers.MaxPooling2D((2, 2))(c1)

    c2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(p1)
    c2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c2)
    p2 = layers.MaxPooling2D((2, 2))(c2)

    c3 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(p2)
    c3 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(c3)
    p3 = layers.MaxPooling2D((2, 2))(c3)

    # Bottleneck
    c4 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(p3)
    c4 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(c4)

    # Decoder
    u5 = layers.Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(c4)
    u5 = layers.concatenate([u5, c3])
    c5 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(u5)
    c5 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(c5)

    u6 = layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = layers.concatenate([u6, c2])
    c6 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(u6)
    c6 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c6)

    u7 = layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = layers.concatenate([u7, c1])
    c7 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(u7)
    c7 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c7)

    outputs = layers.Conv2D(3, (1, 1), activation='sigmoid')(c7)

    model = models.Model(inputs, outputs)
    return model

# Compile the model
model = unet_model()
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
model.summary()

# Data preparation
def load_images_from_folder(folder, target_size=(256, 256)):
    images = []
    for filename in sorted(os.listdir(folder)):
        img_path = os.path.join(folder, filename)
        img = cv2.imread(img_path)
        if img is not None:
            img = cv2.resize(img, target_size)
            img = img / 255.0  # Normalize the image
            images.append(img)
    return np.array(images)

# Define paths
base_dir = 'Dataset/Dataset'
clear_dir = os.path.join(base_dir, 'clear')
reflection_dir = os.path.join(base_dir, 'img with reflection')

# Load datasets
X_train = load_images_from_folder(reflection_dir)
Y_train = load_images_from_folder(clear_dir)

# Check the shapes of loaded data
print(f"Loaded {len(X_train)} images with reflections.")
print(f"Loaded {len(Y_train)} clear images.")

# Ensure dataset size is consistent
assert len(X_train) == len(Y_train), "Mismatch between input and output datasets."

# Split into training and validation sets
split_idx = int(0.8 * len(X_train))
X_train, X_val = X_train[:split_idx], X_train[split_idx:]
Y_train, Y_val = Y_train[:split_idx], Y_train[split_idx:]

print(f"Training samples: {len(X_train)}, Validation samples: {len(X_val)}")

# Training parameters
batch_size = 16
num_epochs = 50
steps_per_epoch = len(X_train) // batch_size
validation_steps = len(X_val) // batch_size

# Custom data generator
def data_generator(X, Y, batch_size):
    while True:
        indices = np.random.permutation(len(X))
        for i in range(0, len(X), batch_size):
            batch_indices = indices[i:i + batch_size]
            X_batch = [X[j] for j in batch_indices]
            Y_batch = [Y[j] for j in batch_indices]
            yield np.array(X_batch), np.array(Y_batch)

# Create generators
train_gen = data_generator(X_train, Y_train, batch_size)
val_gen = data_generator(X_val, Y_val, batch_size)

# Train the model
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=num_epochs,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps
)

# Save the model
model.save('reflection_removal_unet.h5')


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3             │ (None, 256, 256, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_45 (Conv2D)        │ (None, 256, 256, 64)   │          1,792 │ input_layer_3[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_46 (Conv2D)        │ (None, 256, 256, 64)   │         36,928 │ conv2d_45[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_9           │ (None, 128, 128, 64)   │              0 │ conv2d_46[0][0]        │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_47 (Conv2D)        │ (None, 128, 128, 128)  │         73,856 │ max_pooling2d_9[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_48 (Conv2D)        │ (None, 128, 128, 128)  │        147,584 │ conv2d_47[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_10          │ (None, 64, 64, 128)    │              0 │ conv2d_48[0][0]        │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_49 (Conv2D)        │ (None, 64, 64, 256)    │        295,168 │ max_pooling2d_10[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_50 (Conv2D)        │ (None, 64, 64, 256)    │        590,080 │ conv2d_49[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_11          │ (None, 32, 32, 256)    │              0 │ conv2d_50[0][0]        │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_51 (Conv2D)        │ (None, 32, 32, 512)    │      1,180,160 │ max_pooling2d_11[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_52 (Conv2D)        │ (None, 32, 32, 512)    │      2,359,808 │ conv2d_51[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_transpose_9        │ (None, 64, 64, 256)    │        524,544 │ conv2d_52[0][0]        │
│ (Conv2DTranspose)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_9             │ (None, 64, 64, 512)    │              0 │ conv2d_transpose_9[0]… │
│ (Concatenate)             │                        │                │ conv2d_50[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_53 (Conv2D)        │ (None, 64, 64, 256)    │      1,179,904 │ concatenate_9[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_54 (Conv2D)        │ (None, 64, 64, 256)    │        590,080 │ conv2d_53[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_transpose_10  

 Total params: 7,697,475 (29.36 MB)

 Trainable params: 7,697,475 (29.36 MB)

 Non-trainable params: 0 (0.00 B)

Loaded 499 images with reflections.
Loaded 499 clear images.
Training samples: 399, Validation samples: 100
Epoch 1/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 38s 1s/step - accuracy: 0.4199 - loss: 0.0648 - val_accuracy: 0.7223 - val_loss: 0.0223
Epoch 2/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 78s 1s/step - accuracy: 0.6978 - loss: 0.0134 - val_accuracy: 0.6827 - val_loss: 0.0113
Epoch 3/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 15s 637ms/step - accuracy: 0.6882 - loss: 0.0083 - val_accuracy: 0.7324 - val_loss: 0.0096
Epoch 4/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 15s 638ms/step - accuracy: 0.7374 - loss: 0.0091 - val_accuracy: 0.8131 - val_loss: 0.0088
Epoch 5/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 15s 641ms/step - accuracy: 0.7924 - loss: 0.0074 - val_accuracy: 0.7924 - val_loss: 0.0084
Epoch 6/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 15s 643ms/step - accuracy: 0.7656 - loss: 0.0068 - val_accuracy: 0.7592 - val_loss: 0.0075
Epoch 7/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 15s 645ms/step - accuracy: 0.8138 - loss: 0.0055 - val_accuracy: 0.7766 - val_loss: 0.

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
import cv2

# Define U-Net model
def unet_model(input_shape=(256, 256, 3)):
    inputs = layers.Input(input_shape)

    # Encoder
    c1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    c1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c1)
    p1 = layers.MaxPooling2D((2, 2))(c1)

    c2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(p1)
    c2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c2)
    p2 = layers.MaxPooling2D((2, 2))(c2)

    c3 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(p2)
    c3 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(c3)
    p3 = layers.MaxPooling2D((2, 2))(c3)

    # Bottleneck
    c4 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(p3)
    c4 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(c4)

    # Decoder
    u5 = layers.Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(c4)
    u5 = layers.concatenate([u5, c3])
    c5 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(u5)
    c5 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(c5)

    u6 = layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = layers.concatenate([u6, c2])
    c6 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(u6)
    c6 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c6)

    u7 = layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = layers.concatenate([u7, c1])
    c7 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(u7)
    c7 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c7)

    outputs = layers.Conv2D(3, (1, 1), activation='sigmoid')(c7)

    model = models.Model(inputs, outputs)
    return model

# Custom PSNR metric
def psnr_metric(y_true, y_pred):
    return tf.image.psnr(y_true, y_pred, max_val=1.0)

# Compile the model
model = unet_model()
model.compile(optimizer='adam', loss='mean_absolute_error', metrics=[psnr_metric])
model.summary()

# Load and preprocess images
def load_images(folder, target_size=(256, 256)):
    images = []
    for filename in sorted(os.listdir(folder)):
        img_path = os.path.join(folder, filename)
        img = cv2.imread(img_path)
        if img is not None:
            img = cv2.resize(img, target_size)
            img = img / 255.0
            images.append(img)
    return np.array(images)

# Dataset paths
base_dir = 'Dataset/Dataset'
clear_dir = os.path.join(base_dir, 'clear')
reflection_dir = os.path.join(base_dir, 'img with reflection')

X_data = load_images(reflection_dir)
Y_data = load_images(clear_dir)

# Assert data consistency
assert len(X_data) == len(Y_data), "Dataset mismatch."

# Data splitting
split_idx = int(0.8 * len(X_data))
X_train, X_val = X_data[:split_idx], X_data[split_idx:]
Y_train, Y_val = Y_data[:split_idx], Y_data[split_idx:]

# Define dataset using tf.data
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, Y_train))
val_dataset = tf.data.Dataset.from_tensor_slices((X_val, Y_val))

# Data augmentation
def augment(image, label):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_brightness(image, 0.2)
    return image, label

train_dataset = train_dataset.map(augment).batch(16).prefetch(1)
val_dataset = val_dataset.batch(16).prefetch(1)

# Training
EPOCHS = 50
history = model.fit(train_dataset, validation_data=val_dataset, epochs=EPOCHS)

# Save model
model.save('improved_unet_reflection_removal1.h5')


Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_5             │ (None, 256, 256, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_75 (Conv2D)        │ (None, 256, 256, 64)   │          1,792 │ input_layer_5[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_76 (Conv2D)        │ (None, 256, 256, 64)   │         36,928 │ conv2d_75[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_15          │ (None, 128, 128, 64)   │              0 │ conv2d_76[0][0]        │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_77 (Conv2D)        │ (None, 128, 128, 128)  │         73,856 │ max_pooling2d_15[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_78 (Conv2D)        │ (None, 128, 128, 128)  │        147,584 │ conv2d_77[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_16          │ (None, 64, 64, 128)    │              0 │ conv2d_78[0][0]        │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_79 (Conv2D)        │ (None, 64, 64, 256)    │        295,168 │ max_pooling2d_16[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_80 (Conv2D)        │ (None, 64, 64, 256)    │        590,080 │ conv2d_79[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_17          │ (None, 32, 32, 256)    │              0 │ conv2d_80[0][0]        │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_81 (Conv2D)        │ (None, 32, 32, 512)    │      1,180,160 │ max_pooling2d_17[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_82 (Conv2D)        │ (None, 32, 32, 512)    │      2,359,808 │ conv2d_81[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_transpose_15       │ (None, 64, 64, 256)    │        524,544 │ conv2d_82[0][0]        │
│ (Conv2DTranspose)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_15            │ (None, 64, 64, 512)    │              0 │ conv2d_transpose_15[0… │
│ (Concatenate)             │                        │                │ conv2d_80[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_83 (Conv2D)        │ (None, 64, 64, 256)    │      1,179,904 │ concatenate_15[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_84 (Conv2D)        │ (None, 64, 64, 256)    │        590,080 │ conv2d_83[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_transpose_16  

 Total params: 7,697,475 (29.36 MB)

 Trainable params: 7,697,475 (29.36 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 32s 874ms/step - loss: 0.2075 - psnr_metric: 12.7289 - val_loss: 0.2195 - val_psnr_metric: 12.2349
Epoch 2/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 16s 647ms/step - loss: 0.1976 - psnr_metric: 12.9409 - val_loss: 0.2080 - val_psnr_metric: 12.4856
Epoch 3/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 20s 635ms/step - loss: 0.1792 - psnr_metric: 13.6429 - val_loss: 0.1242 - val_psnr_metric: 16.4751
Epoch 4/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 16s 639ms/step - loss: 0.1692 - psnr_metric: 14.1933 - val_loss: 0.0962 - val_psnr_metric: 18.4078
Epoch 5/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 20s 631ms/step - loss: 0.1600 - psnr_metric: 14.7176 - val_loss: 0.1363 - val_psnr_metric: 15.7365
Epoch 6/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 16s 633ms/step - loss: 0.1647 - psnr_metric: 14.4012 - val_loss: 0.1161 - val_psnr_metric: 17.1580
Epoch 7/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 16s 635ms/step - loss: 0.1555 - psnr_metric: 14.8828 - val_loss: 0.1116 - val_psnr_metric: 17.2814
Epoch 8/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 20s

In [ ]:
import shutil
import os

# Path where your model is saved in Colab (for example, 'reflection_removal_unet.h5')
colab_model_path = '/content/improved_unet_reflection_removal1.h5'

# Path to your Google Drive folder
drive_model_folder = '/content/drive/MyDrive/your_model_folder/'

# Ensure the destination folder exists
os.makedirs(drive_model_folder, exist_ok=True)

# Path where the model will be copied in Drive
drive_model_path = os.path.join(drive_model_folder, 'improved_unet_reflection_removal1.h5')

# Copy the model from Colab to Google Drive
shutil.copy(colab_model_path, drive_model_path)

print(f'Model saved to Google Drive at: {drive_model_path}')


Model saved to Google Drive at: /content/drive/MyDrive/your_model_folder/improved_unet_reflection_removal1.h5
